# Tax Charts

## Datasets
- [IRAS' Collection by Tax Type](https://data.gov.sg/datasets/d_21e22578cabce897e8b27801e5596140/view)

- [Tax Rates for Property Tax](https://data.gov.sg/datasets/d_2109ad1eafff52dab388f9bcd8148a35/view)

- [Median Annual Value and Property Tax By Type of HDB](https://data.gov.sg/datasets/d_48143be392f1ed22f0700835212e5a60/view)

- [Consumer Price Index (CPI), 2019 As Base Year, Annual](https://data.gov.sg/datasets/d_dcb352661fb449c4a4c0ab23aa8d6399/view)

- [Tax Rates for Goods and Services Tax](https://data.gov.sg/datasets/d_2e65ed309aa8d449d1bd0c7ef7c7e4da/view)

- [Tax Rates for Individual Income Tax](https://data.gov.sg/datasets/d_f73055c69144d2e7734c28811d3982aa/view)

- [Taxable Individuals by Assessed Income Group](https://data.gov.sg/datasets/d_f394f202534237671d39b17bd3b506ec/view)

## Preamble

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.colors
import plotly.graph_objects as go

import parser

In [ ]:
def plotly_rgb_to_rgba(rgb_value, alpha):
    return f"rgba{rgb_value[3:-1]}, {alpha})"

## IRAS Tax Collection by Type

In [ ]:
iras_df = await parser.parse_datagov_dataset("data/datasets/sg_iras_tax.json")
iras_df = iras_df.drop("_id", axis=1)
iras_df["tax_collected"] = pd.to_numeric(iras_df["tax_collected"], errors="coerce")
iras_df["financial_year"] = pd.to_numeric(iras_df["financial_year"], errors="coerce")
iras_df = iras_df.fillna(0)
iras_df = iras_df.rename({
    "financial_year": "Financial Year",
    "tax_type": "Tax Type",
    "tax_collected": "Tax Collected",
}, axis=1)
iras_df = iras_df[
    iras_df["Tax Type"].isin([
        "Income Tax - Corporate Income Tax",
        "Income Tax - Individual Income Tax",
        "Income Tax - Withholding Tax",
        "Goods and Services Tax",
        "Property Tax",
        "Stamp Duty",
        "Betting Taxes",
        "Estate Duty",
    ])
]
iras_df = iras_df.replace(to_replace="Income Tax - Corporate Income Tax", value="Corporate Income Tax")
iras_df = iras_df.replace(to_replace="Income Tax - Individual Income Tax", value="Individual Income Tax")
iras_df = iras_df.replace(to_replace="Income Tax - Withholding Tax", value="Withholding Tax")
iras_df

In [ ]:
# Make sure colors are specified in RGB.
color_sequence = plotly.colors.convert_colors_to_same_type(px.colors.qualitative.Bold)[0]

In [ ]:
fig = px.bar(
    iras_df,
    x="Financial Year",
    y="Tax Collected",
    color="Tax Type",
    barmode="stack",
    title="Tax Collected by the IRAS by Tax Type",
    color_discrete_sequence=color_sequence,
)

def update_area_color(trace):
    color = plotly_rgb_to_rgba(trace.line.color, 0.2)
    trace.update(
        line=dict(color=color),
        fillcolor=color,
    )

area_fig = px.area(
    iras_df,
    x="Financial Year",
    y="Tax Collected",
    color="Tax Type",
    color_discrete_sequence=color_sequence,
).update_traces(
    yaxis = "y2",
    line=dict(width=0),
    showlegend=False,
).for_each_trace(update_area_color)

for trace in area_fig.data:
    fig.add_trace(trace)

fig.update_yaxes(title="Tax Collected (SGD)")
fig.update_xaxes(tickangle=45, dtick=1)
fig.update_layout(
    yaxis=dict(overlaying="y2"),
)
fig.show()

In [ ]:
with open("data/charts/iras_tax_collection_bar.json", "w") as file:
    file.write(fig.to_json())

## Property Tax Rates

In [ ]:
prop_tax_rates_df = await parser.parse_datagov_dataset("data/datasets/sg_property_tax_rates.json")
prop_tax_rates_df = prop_tax_rates_df.drop("_id", axis=1)
prop_tax_rates_df = prop_tax_rates_df.replace("na", None)
prop_tax_rates_df = prop_tax_rates_df.rename({
    "from": "Year",
    "type_of_property": "Property Type",
    "annual_value": "Annual Value Range",
    "tax_rate": "Tax Rate",
}, axis=1)

for column in prop_tax_rates_df.columns:
    try:
        prop_tax_rates_df[column] = pd.to_numeric(prop_tax_rates_df[column])
    except:
        pass

prop_tax_rates_df["Year"] = prop_tax_rates_df["Year"].apply(
    lambda year_str: int(year_str.split("-")[0])
)

def extract_range_bounds(range_str):
    if range_str is None:
        return None, None

    if range_str.startswith("Amount exceeding "):
        return int(range_str[len("Amount exceeding "):].replace(',', '')), float("inf")

    lower_bound, upper_bound = range_str.split(" - ")
    lower_bound = int(lower_bound.replace(',', ''))
    upper_bound = int(upper_bound.replace(',', ''))
    return lower_bound, upper_bound

prop_tax_rates_df[["Lower Bound", "Upper Bound"]] = (
    prop_tax_rates_df["Annual Value Range"]
        .apply(extract_range_bounds)
        .apply(pd.Series)
)

# Create dummy value for the last bracket
# so that we can draw a line for it.
upp_inf_df = prop_tax_rates_df[
    prop_tax_rates_df["Upper Bound"] == float("inf")
].copy()
upp_inf_df["Lower Bound"] = 150_000 # Dummy value to make the line long

prop_tax_rates_df = (
    pd.concat((prop_tax_rates_df, upp_inf_df))
        .drop(["Upper Bound", "Annual Value Range"], axis=1)
        .sort_values(["Year", "Lower Bound"], axis=0)
)

prop_tax_rates_df = prop_tax_rates_df.dropna()

min_common_year = min(
    min(
        prop_tax_rates_df[
            prop_tax_rates_df["Property Type"] == "Non-Owner-Occupied Residential Properties"
        ]["Year"]
    ), min(
        prop_tax_rates_df[
            prop_tax_rates_df["Property Type"] == "Owner-Occupied Residential Properties"
        ]["Year"]
    )
)

prop_tax_rates_df = prop_tax_rates_df[
    prop_tax_rates_df["Year"] > min_common_year
]

prop_tax_rates_df

In [ ]:
fig = px.scatter(
    prop_tax_rates_df,
    x="Lower Bound",
    y="Tax Rate",
    animation_frame="Year",
    color="Property Type",
    facet_col="Property Type",
    facet_col_spacing=0.08,
    range_x=[0, 150_000],
    range_y=[0, 40],
    title="Property Tax Rates by Annual Value Range",
    width=900,
    height=600,
)

# Change to step line chart.
fig.update_traces(mode="lines", line_shape='hv')
for frame in fig.frames:
    for trace in frame.data:
        trace["mode"] = "lines"
        trace["line_shape"] = "hv"

# Increase the duration of each frame to 2 seconds.
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000

fig.for_each_xaxis(lambda axis: axis.update({"title": "Annual Value (SGD)", "dtick": 10_000}))
fig.for_each_yaxis(lambda axis: axis.update({"title": "Tax Rate (%)"}))

fig.update_layout(
    showlegend=False,
)

fig.show()

In [ ]:
with open("data/charts/property_tax_rates_step_line.json", "w") as file:
    file.write(fig.to_json())

## Median Annual Value and Property Tax by Type of HDB

In [ ]:
hdb_avalue_tax_df = await parser.parse_datagov_dataset("data/datasets/sg_hdb_annual_value_tax.json")
hdb_avalue_tax_df = hdb_avalue_tax_df.drop("_id", axis=1)
hdb_avalue_tax_df = hdb_avalue_tax_df.replace("na", None)
hdb_avalue_tax_df = hdb_avalue_tax_df.rename({
    "financial_year": "Financial Year",
    "type_of_hdb": "HDB Type",
    "no_of_cases": "Number of Cases",
    "median_annual_value": "Median Annual Value",
    "property_tax_collection": "Property Tax Collection",
}, axis=1)

for column in hdb_avalue_tax_df.columns:
    try:
        hdb_avalue_tax_df[column] = pd.to_numeric(hdb_avalue_tax_df[column])
    except:
        pass

hdb_avalue_tax_df = hdb_avalue_tax_df[hdb_avalue_tax_df["Financial Year"] >= 2014]

hdb_avalue_tax_df["Tax Annual Value Text"] = hdb_avalue_tax_df.apply(
    lambda row: (
        f"<b>Tax:</b> {row["Property Tax Collection"]}<br>"
        f"<b>AV:</b> {row["Median Annual Value"]}<br>"
        f"<b>Cases:</b> {row["Number of Cases"]}"
    ),
    axis=1
)

hdb_avalue_tax_df

In [ ]:
fig = px.scatter(
    hdb_avalue_tax_df,
    x="Median Annual Value",
    y="Property Tax Collection",
    color="HDB Type",
    size="Number of Cases",
    text="Tax Annual Value Text",
    hover_data=["Number of Cases"],
    animation_frame="Financial Year",
    title="Property Tax Collection Against Annual Value by Year and HDB Type",
    range_x=[5_000, 20_000],
    range_y=[-10_000, 120_000],
    width=800,
)

# Border for all points.
fig.update_traces(marker=dict(
    line=dict(width=1, color='black')
))

# Set text position.
fig.update_traces(
    textposition="middle right",
    textfont=dict(color="black"),
)

fig.update_traces(
    selector=dict(name="4 Room"),
    textposition="middle left",
    textfont=dict(color="black"),
)

fig.update_layout(
    xaxis_title="Median Annual Value (SGD)",
    yaxis_title="Property Tax Collection (SGD)",
)

fig.show()

In [ ]:
with open("data/charts/property_tax_collection_annual_value_bubble.json", "w") as file:
    file.write(fig.to_json())

## CPI  Against GST

In [ ]:
cpi_a_df = await parser.parse_datagov_dataset("data/datasets/sg_cpi_annual.json")
cpi_a_df = cpi_a_df.drop("_id", axis=1) # _id columns is redundant since Pandas maintains an index.
cpi_a_df = cpi_a_df.replace("na", None)

numeric_cols = cpi_a_df.columns.drop("DataSeries")
cpi_a_df[numeric_cols] = cpi_a_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

cpi_a_gst_df = cpi_a_df.copy(deep=True)
cpi_a_gst_df = cpi_a_gst_df[
    cpi_a_gst_df["DataSeries"].isin([
        "    Food",
        "    Clothing & Footwear",
        "    Household Durables & Services",
        "    Recreation & Culture",
        "    Miscellaneous Goods & Services"
    ])
]
cpi_a_gst_df = cpi_a_gst_df[["DataSeries"] + [str(year) for year in range(2000, 2025)]]
cpi_a_gst_df = cpi_a_gst_df.set_index("DataSeries").stack().reset_index()
cpi_a_gst_df = cpi_a_gst_df.rename({"DataSeries": "Category", "level_1": "Year", 0: "CPI"}, axis=1)
cpi_a_gst_df

In [ ]:
gst_df = await parser.parse_datagov_dataset("data/datasets/sg_gst.json")
gst_df = gst_df.drop("_id", axis=1)
gst_df["effective_from"] = gst_df["effective_from"].apply(lambda date: int(date.split("-")[0])) # Extract year
gst_df["tax_rate"] = pd.to_numeric(gst_df["tax_rate"])
gst_df = gst_df.rename({"effective_from": "Year", "tax_rate": "Tax Rate"}, axis=1)
gst_df

In [ ]:
cpi_fig = px.bar(
    cpi_a_gst_df,
    x="Year",
    y="CPI",
    color="Category",
    barmode="stack",
    color_discrete_sequence=px.colors.qualitative.Set2,
)

gst_fig = px.line(
    gst_df,
    x="Year",
    y="Tax Rate",
).update_traces(
    yaxis = "y2",
    mode="lines+markers",
    opacity=0.8,
    line=dict(width=4, color="#444444"),
    marker=dict(symbol="circle", size=12),
    name="GST Rate",
    showlegend=True,
)

for trace in gst_fig.data:
    cpi_fig.add_trace(trace)

cpi_fig.update_layout(
    title="CPI by Category Against GST Rate",
    yaxis=dict(title="CPI"),
    yaxis2=dict(title="GST Rate (%)", overlaying="y", side="right"),
    legend=dict(
        title="Legend",
        x=1.05,
    ),
    width=1_250,
)

cpi_fig.update_xaxes(tickangle=45, dtick=1)

cpi_fig.show()

In [ ]:
with open("data/charts/cpi_vs_gst_line_bar.json", "w") as file:
    file.write(cpi_fig.to_json())

## Income Tax Rates

In [ ]:
income_tax_rates_df = await parser.parse_datagov_dataset("data/datasets/sg_income_tax_rates.json")
income_tax_rates_df = income_tax_rates_df.drop("_id", axis=1)
income_tax_rates_df = income_tax_rates_df.replace("na", None)
income_tax_rates_df = income_tax_rates_df.rename({
    "from": "Year",
    "chargeable_income": "Chargeable Income",
    "tax_rate": "Tax Rate",
}, axis=1)

for column in income_tax_rates_df.columns:
    try:
        income_tax_rates_df[column] = pd.to_numeric(income_tax_rates_df[column])
    except:
        pass

def extract_range_bounds(range_str):
    if range_str is None:
        return None, None

    if range_str.startswith("More than "):
        return int(range_str[len("More than "):].replace(',', '')), float("inf")

    # One of the rows uses "to" instead of "-".
    delim = "to" if "to" in range_str else "-"
    lower_bound, upper_bound = range_str.split(delim)
    lower_bound = int(lower_bound.replace(',', ''))
    upper_bound = int(upper_bound.replace(',', ''))
    return lower_bound, upper_bound

income_tax_rates_df[["Lower Bound", "Upper Bound"]] = (
    income_tax_rates_df["Chargeable Income"]
        .apply(extract_range_bounds)
        .apply(pd.Series)
)

# Create dummy value for the last bracket
# so that we can draw a line for it.
upp_inf_df = income_tax_rates_df[
    income_tax_rates_df["Upper Bound"] == float("inf")
].copy()
upp_inf_df["Lower Bound"] = 1_250_000 # Dummy value to make the line long

income_tax_rates_df = (
    pd.concat((income_tax_rates_df, upp_inf_df))
        .drop(["Upper Bound", "Chargeable Income"], axis=1)
        .sort_values(["Year", "Lower Bound"], axis=0)
)

# Include 2012 and above.
income_tax_rates_df = income_tax_rates_df[income_tax_rates_df["Year"] >= 2012]

income_tax_rates_df = income_tax_rates_df.dropna()
income_tax_rates_df

In [ ]:
fig = px.scatter(
    income_tax_rates_df,
    x="Lower Bound",
    y="Tax Rate",
    animation_frame="Year",
    range_x=[0, max(income_tax_rates_df["Lower Bound"])],
    range_y=[0, max(income_tax_rates_df["Tax Rate"]) + 5],
    title="Income Tax Rates by Yearly Income Range",
    width=600,
)

# Change to step line chart.
fig.update_traces(mode="lines", line_shape='hv')
for frame in fig.frames:
    for trace in frame.data:
        trace["mode"] = "lines"
        trace["line_shape"] = "hv"

# Increase the duration of each frame to 2 seconds.
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000

fig.for_each_xaxis(lambda axis: axis.update({"title": "Yearly Income (SGD)", "dtick": 50_000}))
fig.for_each_yaxis(lambda axis: axis.update({"title": "Tax Rate (%)"}))

fig.update_layout(
    showlegend=False,
)

fig.show()

In [ ]:
with open("data/charts/income_tax_rates_step_line.json", "w") as file:
    file.write(fig.to_json())

## Individual Income Tax

In [ ]:
ind_income_tax_df = await parser.parse_datagov_dataset("data/datasets/sg_individual_income_tax.json")
ind_income_tax_df = ind_income_tax_df.drop("_id", axis=1)
ind_income_tax_df = ind_income_tax_df.replace("na", None)

ind_income_tax_df = ind_income_tax_df.rename({
    "year_of_assessment": "Year of Assessment",
    "assessed_income_group": "Assessed Income Group",
    "resident_type": "Resident Type",
    "number_of_taxpayers": "Number of Tax Payers",
    "assessable_income": "Assessable Income",
    "chargeable_income": "Chargeable Income",
    "net_tax_assessed": "Net Tax Assessed",
}, axis=1)

for column in ind_income_tax_df.columns:
    try:
        ind_income_tax_df[column] = pd.to_numeric(ind_income_tax_df[column])
    except:
        pass

ind_income_tax_df = ind_income_tax_df[ind_income_tax_df["Year of Assessment"] >= 2014]

ind_income_tax_df = ind_income_tax_df[ind_income_tax_df["Resident Type"] == "Tax Resident"]
ind_income_tax_df = ind_income_tax_df.drop("Resident Type", axis=1)

def extract_range_bounds(range_str):
    if range_str is None:
        return None, None

    if " & below" in range_str:
        return 0, int(range_str[:-len(" & below")].replace(',', ''))
        
    if " & above" in range_str:
        return int(range_str[:-len(" & above")].replace(',', '')), float("inf")

    lower_bound, upper_bound = range_str.split("-")
    lower_bound = int(lower_bound.replace(',', ''))
    upper_bound = int(upper_bound.replace(',', ''))
    return lower_bound, upper_bound

ind_income_tax_df[["Lower Bound", "Upper Bound"]] = (
    ind_income_tax_df["Assessed Income Group"]
        .apply(extract_range_bounds)
        .apply(pd.Series)
)

ind_income_tax_df["Tax-Income Percentage"] = (ind_income_tax_df["Net Tax Assessed"] / ind_income_tax_df["Assessable Income"]) * 100
ind_income_tax_df = ind_income_tax_df.fillna(0)

ind_income_tax_df = ind_income_tax_df.sort_values(["Year of Assessment", "Upper Bound"])
ind_income_tax_df

In [ ]:
# "Upper Bound" is added as an index to sort the data frame. It is removed immediately after.
ind_income_tax_net_tax_df = ind_income_tax_df.pivot(
    index=["Upper Bound", "Assessed Income Group"],
    columns="Year of Assessment",
    values="Net Tax Assessed"
).sort_values("Upper Bound", ascending=False)
ind_income_tax_net_tax_df = ind_income_tax_net_tax_df.reset_index("Upper Bound", drop=True)
ind_income_tax_net_tax_df

In [ ]:
# "Upper Bound" is added as an index to sort the data frame. It is removed immediately after.
ind_income_tax_assessable_income_df = ind_income_tax_df.pivot(
    index=["Upper Bound", "Assessed Income Group"],
    columns="Year of Assessment",
    values="Assessable Income"
).sort_values("Upper Bound", ascending=False)
ind_income_tax_assessable_income_df = ind_income_tax_assessable_income_df.reset_index("Upper Bound", drop=True)
ind_income_tax_assessable_income_df

In [ ]:
# Weird shenanigans.
customdata = np.stack(
    (
        ind_income_tax_assessable_income_df[::-1].transpose().to_numpy().flatten(),
        ind_income_tax_net_tax_df[::-1].transpose().to_numpy().flatten(),
    ),
    axis=-1
)

In [ ]:
fig = go.Figure(
    layout=dict(
        title="Percentage of Assessed Income Paid in Taxes by Assessed Income Group from 2014 to 2023",
        xaxis=dict(title=dict(text="Year of Assessment")),
        yaxis=dict(title=dict(text="Assessed Income Group (S$)")),
        width=1200,
    )
)

fig.add_trace(
    go.Heatmap(
        x = ind_income_tax_df["Year of Assessment"],
        y = ind_income_tax_df["Assessed Income Group"],
        z = ind_income_tax_df["Tax-Income Percentage"],
        colorscale="Viridis",
        colorbar=dict(
            title="Percentage of Assessed Income Paid in Taxes",
            tickvals=[0, 5, 10, 15, 20],
            ticktext=["0%", "5%", "10%", "15%", "20%"],
        ),
        customdata=customdata,
        hovertemplate = (
            "Year of Assessment: %{x}<br>"
            "Assessed Income Group: %{y}<br>"
            "Percentage of Assessed Income Paid in Taxes: %{z}<br>"
            "Assessable Income: %{customdata[0]}<br>"
            "Net Tax Assessed: %{customdata[1]}<br>"
            "<extra></extra>"
        ),
    )
)

fig.update_xaxes(dtick=1)

fig

In [ ]:
with open("data/charts/income_tax_heatmap.json", "w") as file:
    file.write(fig.to_json())